In [1]:
!pip install py2neo

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 6.2 MB/s eta 0:00:00


In [3]:
import os
import csv
from py2neo import Graph, Node, Relationship

# 配置 Neo4j 连接
NEO4J_URI = "neo4j+s://33daa25d.databases.neo4j.io"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "oykvwpgg99SOby9BLlbWsjQ14QWMQh_VrLkY-bMdCMU"

# 创建图实例
graph = Graph(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# 清空图数据（可选）
graph.delete_all()

# 映射：文件名 → 主键字段
NODE_KEYS = {
    'supplier_nodes.csv': 'supplierId:ID(Supplier)',
    'shipper_nodes.csv': 'shipperId:ID(Shipper)',
    'review_nodes.csv': 'reviewId:ID(Review)',
    'product_nodes.csv': 'productId:ID(Product)',
    'employee_nodes.csv': 'employeeId:ID(Employee)',
    'customer_nodes.csv': 'customerId:ID(Customer)',
    'category_nodes.csv': 'categoryId:ID(Category)'
}

# 映射：文件名 → 标签字段
LABEL_KEYS = {
    'supplier_nodes.csv': 'labels:LABEL',
    'shipper_nodes.csv': 'labels:LABEL',
    'review_nodes.csv': 'labels:LABEL',
    'product_nodes.csv': 'labels:LABEL',
    'employee_nodes.csv': 'labels:LABEL',
    'customer_nodes.csv': 'labels:LABEL',
    'category_nodes.csv': 'labels:LABEL'
}

# 映射：边文件 → 边类型
EDGE_TYPES = {
    'review_product_edges.csv': ('Review', 'Product', 'ABOUT'),
    'product_supplier_edges.csv': ('Product', 'Supplier', 'SUPPLIED_BY'),
    'customer_order_edges.csv': ('Customer', 'Order', 'PLACED'),
    'order_shipper_edges.csv': ('Order', 'Shipper', 'SHIPPED_VIA'),
    'order_product_edges.csv': ('Order', 'Product', 'CONTAINS'),
    'product_category_edges.csv': ('Product', 'Category', 'BELONGS_TO'),
    'customer_review_edges.csv': ('Customer', 'Review', 'WROTE'),
    'employee_reports_to_edges.csv': ('Employee', 'Employee', 'REPORTS_TO'),
    'employee_order_edges.csv': ('Employee', 'Order', 'PROCESSED')
}

# 创建节点
def create_nodes_from_csv(filepath, primary_key, label_key):
    filename = os.path.basename(filepath)
    print(f"导入节点：{filename}")
    with open(filepath, encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            node_id = row.pop(primary_key)
            labels = row.pop(label_key).split(',')
            node = Node(*labels, id=node_id, **{k: v for k, v in row.items() if v})
            graph.create(node)

# 创建边
def create_edges_from_csv(filepath, start_label, end_label, rel_type):
    filename = os.path.basename(filepath)
    print(f"导入边：{filename}")
    with open(filepath, encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            start_id = row[':START_ID(' + start_label + ')']
            end_id = row[':END_ID(' + end_label + ')']
            start_node = graph.nodes.match(start_label, id=start_id).first()
            end_node = graph.nodes.match(end_label, id=end_id).first()
            if start_node and end_node:
                rel = Relationship(start_node, rel_type, end_node)
                graph.create(rel)

# 遍历当前目录下的所有 CSV 文件
for file in os.listdir('.'):
    if not file.endswith('.csv'):
        continue
    if 'nodes' in file:
        key = NODE_KEYS.get(file)
        label_key = LABEL_KEYS.get(file)
        if key:
            create_nodes_from_csv(file, key, label_key)
    elif 'edges' in file:
        info = EDGE_TYPES.get(file)
        if info:
            create_edges_from_csv(file, *info)

print("✅ 所有数据已成功导入 Neo4j！")

导入节点：category_nodes.csv
导入节点：customer_nodes.csv
导入边：customer_order_edges.csv
导入边：customer_review_edges.csv
导入节点：employee_nodes.csv
导入边：employee_order_edges.csv
导入边：employee_reports_to_edges.csv
导入边：order_product_edges.csv
导入边：order_shipper_edges.csv
导入边：product_category_edges.csv
导入节点：product_nodes.csv
导入边：product_supplier_edges.csv
导入节点：review_nodes.csv
导入边：review_product_edges.csv
导入节点：shipper_nodes.csv
导入节点：supplier_nodes.csv
✅ 所有数据已成功导入 Neo4j！


In [16]:
from neo4j import GraphDatabase
import csv
import os
from datetime import datetime

class Neo4jImporter:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def close(self):
        self.driver.close()
    
    def import_nodes(self, file_path, label):
        with self.driver.session() as session:
            with open(file_path, 'r', encoding='utf-8') as file:
                reader = csv.DictReader(file)
                batch = []
                batch_size = 100
                
                for row in reader:
                    # 准备节点属性
                    properties = {k: v for k, v in row.items()}
                    
                    # 特殊处理日期字段
                    if 'OrderDate' in properties and properties['OrderDate']:
                        properties['OrderDate'] = datetime.strptime(properties['OrderDate'], '%Y-%m-%d %H:%M:%S')
                    if 'ShippedDate' in properties and properties['ShippedDate']:
                        properties['ShippedDate'] = datetime.strptime(properties['ShippedDate'], '%Y-%m-%d %H:%M:%S')
                    if 'RequiredDate' in properties and properties['RequiredDate']:
                        properties['RequiredDate'] = datetime.strptime(properties['RequiredDate'], '%Y-%m-%d %H:%M:%S')
                    if 'BirthDate' in properties and properties['BirthDate']:
                        properties['BirthDate'] = datetime.strptime(properties['BirthDate'], '%Y-%m-%d')
                    if 'HireDate' in properties and properties['HireDate']:
                        properties['HireDate'] = datetime.strptime(properties['HireDate'], '%Y-%m-%d')
                    
                    batch.append(properties)
                    
                    if len(batch) >= batch_size:
                        session.execute_write(self._create_nodes, batch, label)
                        batch = []
                
                if batch:
                    session.execute_write(self._create_nodes, batch, label)
    
    @staticmethod
    def _create_nodes(tx, batch, label):
        query = f"""
        UNWIND $batch AS row
        CREATE (n:{label})
        SET n = row
        """
        tx.run(query, batch=batch)
    
    def import_relations(self, file_path, rel_type, start_label, end_label, start_id_prop, end_id_prop, extra_props=None):
        with self.driver.session() as session:
            with open(file_path, 'r', encoding='utf-8') as file:
                reader = csv.DictReader(file)
                batch = []
                batch_size = 100
                
                for row in reader:
                    # 准备关系属性
                    rel_properties = {}
                    if extra_props:
                        for prop in extra_props:
                            if prop in row:
                                rel_properties[prop] = row[prop]
                    
                    # 特殊处理数值类型
                    if 'Rating' in row:
                        rel_properties['Rating'] = float(row['Rating'])
                    if 'UnitPrice' in row:
                        rel_properties['UnitPrice'] = float(row['UnitPrice'])
                    if 'Quantity' in row:
                        rel_properties['Quantity'] = int(row['Quantity'])
                    if 'Discount' in row:
                        rel_properties['Discount'] = float(row['Discount'])
                    if 'Freight' in row:
                        rel_properties['Freight'] = float(row['Freight'])
                    
                    # 创建关系对象
                    relation = {
                        'start_id': row[':START_ID'],
                        'end_id': row[':END_ID'],
                        'properties': rel_properties
                    }
                    batch.append(relation)
                    
                    if len(batch) >= batch_size:
                        session.execute_write(
                            self._create_relations, 
                            batch, 
                            rel_type, 
                            start_label, 
                            end_label,
                            start_id_prop,
                            end_id_prop
                        )
                        batch = []
                
                if batch:
                    session.execute_write(
                        self._create_relations, 
                        batch, 
                        rel_type, 
                        start_label, 
                        end_label,
                        start_id_prop,
                        end_id_prop
                    )
    
    @staticmethod
    def _create_relations(tx, batch, rel_type, start_label, end_label, start_id_prop, end_id_prop):
        query = f"""
        UNWIND $batch AS rel
        MATCH (start:{start_label} {{{start_id_prop}: rel.start_id}})
        MATCH (end:{end_label} {{{end_id_prop}: rel.end_id}})
        CREATE (start)-[r:{rel_type}]->(end)
        SET r = rel.properties
        """
        tx.run(query, batch=batch)

def main():
    # Neo4j数据库连接配置
    uri = "neo4j+s://981d8a45.databases.neo4j.io"  # 修改为你的Neo4j地址
    user = "neo4j"                # 修改为你的用户名
    password = "7bhGR2TUaVYg3N72GVGSf2g8DP16R-tmOXa4Vid0PFw"         # 修改为你的密码
    
    # 文件路径配置（假设所有文件在当前目录）
    file_dir = './'
    files = {
        'nodes': {
            'supplier': ('supplier_nodes.csv', 'Supplier'),
            'shipper': ('shipper_nodes.csv', 'Shipper'),
            'review': ('review_nodes.csv', 'Review'),
            'product': ('product_nodes.csv', 'Product'),
            'order': ('order_nodes.csv', 'Order'),
            'employee': ('employee_nodes.csv', 'Employee'),
            'customer': ('customer_nodes.csv', 'Customer'),
            'category': ('category_nodes.csv', 'Category'),
        },
        'relations': {
            'review_product': {
                'file': 'review_product_edges.csv',
                'rel_type': 'ABOUT',
                'start_label': 'Review',
                'end_label': 'Product',
                'start_id': 'reviewId',
                'end_id': 'productId',
                'extra_props': None
            },
            'product_supplier': {
                'file': 'product_supplier_edges.csv',
                'rel_type': 'SUPPLIED_BY',
                'start_label': 'Product',
                'end_label': 'Supplier',
                'start_id': 'productId',
                'end_id': 'supplierId',
                'extra_props': None
            },
            'customer_order': {
                'file': 'customer_order_edges.csv',
                'rel_type': 'PLACED',
                'start_label': 'Customer',
                'end_label': 'Order',
                'start_id': 'customerId',
                'end_id': 'orderId',
                'extra_props': None
            },
            'product_category': {
                'file': 'product_category_edges.csv',
                'rel_type': 'BELONGS_TO',
                'start_label': 'Product',
                'end_label': 'Category',
                'start_id': 'productId',
                'end_id': 'categoryId',
                'extra_props': None
            },
            'order_shipper': {
                'file': 'order_shipper_edges.csv',
                'rel_type': 'SHIPPED_VIA',
                'start_label': 'Order',
                'end_label': 'Shipper',
                'start_id': 'orderId',
                'end_id': 'shipperId',
                'extra_props': None
            },
            'order_product': {
                'file': 'order_product_edges.csv',
                'rel_type': 'CONTAINS',
                'start_label': 'Order',
                'end_label': 'Product',
                'start_id': 'orderId',
                'end_id': 'productId',
                'extra_props': ['UnitPrice', 'Quantity', 'Discount']
            },
            'employee_reports': {
                'file': 'employee_reports_to_edges.csv',
                'rel_type': 'REPORTS_TO',
                'start_label': 'Employee',
                'end_label': 'Employee',
                'start_id': 'employeeId',
                'end_id': 'employeeId',
                'extra_props': None
            },
            'employee_order': {
                'file': 'employee_order_edges.csv',
                'rel_type': 'PROCESSED',
                'start_label': 'Employee',
                'end_label': 'Order',
                'start_id': 'employeeId',
                'end_id': 'orderId',
                'extra_props': None
            },
            'customer_review': {
                'file': 'customer_review_edges.csv',
                'rel_type': 'WROTE',
                'start_label': 'Customer',
                'end_label': 'Review',
                'start_id': 'customerId',
                'end_id': 'reviewId',
                'extra_props': None
            }
        }
    }

    importer = Neo4jImporter(uri, user, password)
    
    try:
        # 导入所有节点
        print("开始导入节点...")
        for key, (file_name, label) in files['nodes'].items():
            file_path = os.path.join(file_dir, file_name)
            print(f"导入 {file_name} 作为 {label} 节点")
            importer.import_nodes(file_path, label)
        
        # 导入所有关系
        print("\n开始导入关系...")
        for key, config in files['relations'].items():
            file_path = os.path.join(file_dir, config['file'])
            print(f"导入 {config['file']} 关系: {config['rel_type']}")
            importer.import_relations(
                file_path,
                config['rel_type'],
                config['start_label'],
                config['end_label'],
                config['start_id'],
                config['end_id'],
                config['extra_props']
            )
        
        print("\n数据导入完成！")
    
    except Exception as e:
        print(f"导入过程中出错: {str(e)}")
    finally:
        importer.close()

if __name__ == "__main__":
    main()

开始导入节点...
导入 supplier_nodes.csv 作为 Supplier 节点
导入 shipper_nodes.csv 作为 Shipper 节点
导入 review_nodes.csv 作为 Review 节点
导入 product_nodes.csv 作为 Product 节点
导入 order_nodes.csv 作为 Order 节点
导入 employee_nodes.csv 作为 Employee 节点
导入 customer_nodes.csv 作为 Customer 节点
导入 category_nodes.csv 作为 Category 节点

开始导入关系...
导入 review_product_edges.csv 关系: ABOUT
导入过程中出错: ':START_ID'


In [19]:
from py2neo import Graph, Node, Relationship, Subgraph
import pandas as pd
import os

# 连接 Neo4j
uri = "neo4j+s://981d8a45.databases.neo4j.io"
user = "neo4j"
password = "7bhGR2TUaVYg3N72GVGSf2g8DP16R-tmOXa4Vid0PFw"  # 替换为你的 Neo4j 密码
graph = Graph(uri, auth=(user, password))

# 清空数据库
graph.delete_all()

# 设置 CSV 文件路径（假设与脚本同目录）
csv_dir = os.path.dirname(".")

# 映射文件名到节点标签
node_files = {
    "supplier_nodes.csv": "Supplier",
    "shipper_nodes.csv": "Shipper",
    "review_nodes.csv": "Review",
    "product_nodes.csv": "Product",
    "employee_nodes.csv": "Employee",
    "customer_nodes.csv": "Customer",
    "category_nodes.csv": "Category"
}

# 创建节点
for filename, label in node_files.items():
    path = os.path.join(csv_dir, filename)
    if not os.path.exists(path):
        print(f"⚠️ 文件不存在: {path}")
        continue
    df = pd.read_csv(path)
    df = df.fillna('')  # 处理空值
    nodes = []
    for _, row in df.iterrows():
        node = Node(label)
        for col in df.columns:
            if col == "labels:LABEL" or col.endswith(":LABEL"):
                continue
            key = col.split(":")[0]
            node[key] = str(row[col])
        nodes.append(node)
    graph.create(Subgraph(nodes))
    print(f"✅ 已导入 {len(nodes)} 个 {label} 节点")

# 创建关系（所有 edges 文件）
edges_files = [
    "review_product_edges.csv",
    "product_supplier_edges.csv",
    "customer_order_edges.csv",
    "product_category_edges",
    "order_shipper_edges.csv",
    "order_product_edges.csv",
    "employee_reports_to_edges.csv",
    "employee_order_edges.csv",
    "customer_review_edges.csv"
]

for filename in edges_files:
    path = os.path.join(csv_dir, filename)
    if not os.path.exists(path):
        print(f"⚠️ 文件不存在: {path}")
        continue
    df = pd.read_csv(path)
    df = df.fillna('')
    for _, row in df.iterrows():
        start_id = str(row[":START_ID"])
        end_id = str(row[":END_ID"])
        rel_type = row[":TYPE"]

        # 查找起始和结束节点
        start_node = None
        end_node = None

        # 根据 ID 查找节点（假设 ID 字段为 productId, supplierId 等）
        for label in ["Product", "Supplier", "Review", "Order", "Customer", "Employee", "Category", "Shipper"]:
            if start_node is None:
                start_node = graph.nodes.match(label).where(f"_.{label.lower()}Id = '{start_id}'").first()
            if end_node is None:
                end_node = graph.nodes.match(label).where(f"_.{label.lower()}Id = '{end_id}'").first()

        if start_node and end_node:
            rel = Relationship(start_node, rel_type, end_node)
            graph.create(rel)
        else:
            print(f"⚠️ 无法创建关系: {start_id} -[{rel_type}]-> {end_id}")

    print(f"✅ 已处理关系文件: {filename}")

print("🎉 所有数据导入完成！")

✅ 已导入 15 个 Supplier 节点
✅ 已导入 10 个 Shipper 节点
✅ 已导入 30 个 Review 节点
✅ 已导入 10 个 Product 节点
✅ 已导入 3 个 Employee 节点
✅ 已导入 20 个 Customer 节点
✅ 已导入 20 个 Category 节点


KeyError: ':START_ID'

In [20]:
from neo4j import GraphDatabase
import csv
import os
from datetime import datetime
import logging

# 配置日志
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class Neo4jImporter:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def close(self):
        self.driver.close()
    
    def clean_database(self):
        """删除数据库中的所有节点和关系"""
        with self.driver.session() as session:
            query = "MATCH (n) DETACH DELETE n"
            session.run(query)
            logger.info("数据库已清空")
    
    def import_nodes(self, file_path, label):
        """导入节点数据"""
        with self.driver.session() as session:
            with open(file_path, 'r', encoding='utf-8') as file:
                reader = csv.DictReader(file)
                batch = []
                batch_size = 100
                
                for row in reader:
                    # 准备节点属性
                    properties = {}
                    for key, value in row.items():
                        # 处理特殊字段
                        if key == "BirthDate" or key == "HireDate":
                            properties[key] = datetime.strptime(value, '%Y-%m-%d').date()
                        elif key in ["OrderDate", "RequiredDate", "ShippedDate"]:
                            if value:  # 处理空日期
                                properties[key] = datetime.strptime(value, '%Y-%m-%d %H:%M:%S')
                        else:
                            properties[key] = value
                    
                    batch.append(properties)
                    
                    if len(batch) >= batch_size:
                        session.execute_write(self._create_nodes, batch, label)
                        batch = []
                
                if batch:
                    session.execute_write(self._create_nodes, batch, label)
                logger.info(f"导入节点: {file_path} -> {label} ({reader.line_num} 行)")
    
    @staticmethod
    def _create_nodes(tx, batch, label):
        query = f"""
        UNWIND $batch AS row
        CREATE (n:{label})
        SET n = row
        """
        tx.run(query, batch=batch)
    
    def import_relations(self, file_path, rel_type, start_label, end_label, 
                         start_id_prop, end_id_prop, extra_props=None):
        """导入关系数据"""
        with self.driver.session() as session:
            with open(file_path, 'r', encoding='utf-8') as file:
                reader = csv.DictReader(file)
                batch = []
                batch_size = 100
                
                for row in reader:
                    # 准备关系属性
                    rel_properties = {}
                    if extra_props:
                        for prop in extra_props:
                            if prop in row:
                                # 转换数值类型
                                if prop in ["UnitPrice", "Discount", "Freight", "Rating"]:
                                    rel_properties[prop] = float(row[prop])
                                elif prop in ["Quantity"]:
                                    rel_properties[prop] = int(row[prop])
                                else:
                                    rel_properties[prop] = row[prop]
                    
                    # 创建关系对象
                    relation = {
                        'start_id': row[':START_ID'],
                        'end_id': row[':END_ID'],
                        'properties': rel_properties
                    }
                    batch.append(relation)
                    
                    if len(batch) >= batch_size:
                        session.execute_write(
                            self._create_relations, 
                            batch, 
                            rel_type, 
                            start_label, 
                            end_label,
                            start_id_prop,
                            end_id_prop
                        )
                        batch = []
                
                if batch:
                    session.execute_write(
                        self._create_relations, 
                        batch, 
                        rel_type, 
                        start_label, 
                        end_label,
                        start_id_prop,
                        end_id_prop
                    )
                logger.info(f"导入关系: {file_path} -> {rel_type} ({reader.line_num} 行)")
    
    @staticmethod
    def _create_relations(tx, batch, rel_type, start_label, end_label, start_id_prop, end_id_prop):
        query = f"""
        UNWIND $batch AS rel
        MATCH (start:{start_label} {{{start_id_prop}: rel.start_id}})
        MATCH (end:{end_label} {{{end_id_prop}: rel.end_id}})
        CREATE (start)-[r:{rel_type}]->(end)
        SET r = rel.properties
        """
        tx.run(query, batch=batch)

def main():
    # Neo4j数据库连接配置
    uri = "neo4j+s://981d8a45.databases.neo4j.io"  # 修改为你的Neo4j地址
    user = "neo4j"                # 修改为你的用户名
    password = "7bhGR2TUaVYg3N72GVGSf2g8DP16R-tmOXa4Vid0PFw"         # 修改为你的密码
    
    # 文件路径配置（假设所有文件在当前目录）
    file_dir = './'
    
    # 节点文件配置
    node_files = [
        ('supplier_nodes.csv', 'Supplier'),
        ('shipper_nodes.csv', 'Shipper'),
        ('review_nodes.csv', 'Review'),
        ('product_nodes.csv', 'Product'),
        ('order_nodes.csv', 'Order'),
        ('employee_nodes.csv', 'Employee'),
        ('customer_nodes.csv', 'Customer'),
        ('category_nodes.csv', 'Category'),
    ]
    
    # 关系文件配置
    relation_files = [
        {
            'file': 'review_product_edges.csv',
            'rel_type': 'ABOUT',
            'start_label': 'Review',
            'end_label': 'Product',
            'start_id': 'reviewId',
            'end_id': 'productId',
            'extra_props': None
        },
        {
            'file': 'product_supplier_edges.csv',
            'rel_type': 'SUPPLIED_BY',
            'start_label': 'Product',
            'end_label': 'Supplier',
            'start_id': 'productId',
            'end_id': 'supplierId',
            'extra_props': None
        },
        {
            'file': 'customer_order_edges.csv',
            'rel_type': 'PLACED',
            'start_label': 'Customer',
            'end_label': 'Order',
            'start_id': 'customerId',
            'end_id': 'orderId',
            'extra_props': None
        },
        {
            'file': 'product_category_edges.csv',
            'rel_type': 'BELONGS_TO',
            'start_label': 'Product',
            'end_label': 'Category',
            'start_id': 'productId',
            'end_id': 'categoryId',
            'extra_props': None
        },
        {
            'file': 'order_shipper_edges.csv',
            'rel_type': 'SHIPPED_VIA',
            'start_label': 'Order',
            'end_label': 'Shipper',
            'start_id': 'orderId',
            'end_id': 'shipperId',
            'extra_props': None
        },
        {
            'file': 'order_product_edges.csv',
            'rel_type': 'CONTAINS',
            'start_label': 'Order',
            'end_label': 'Product',
            'start_id': 'orderId',
            'end_id': 'productId',
            'extra_props': ['UnitPrice', 'Quantity', 'Discount']
        },
        {
            'file': 'employee_reports_to_edges.csv',
            'rel_type': 'REPORTS_TO',
            'start_label': 'Employee',
            'end_label': 'Employee',
            'start_id': 'employeeId',
            'end_id': 'employeeId',
            'extra_props': None
        },
        {
            'file': 'employee_order_edges.csv',
            'rel_type': 'PROCESSED',
            'start_label': 'Employee',
            'end_label': 'Order',
            'start_id': 'employeeId',
            'end_id': 'orderId',
            'extra_props': None
        },
        {
            'file': 'customer_review_edges.csv',
            'rel_type': 'WROTE',
            'start_label': 'Customer',
            'end_label': 'Review',
            'start_id': 'customerId',
            'end_id': 'reviewId',
            'extra_props': None
        }
    ]

    importer = Neo4jImporter(uri, user, password)
    
    try:
        # 清空数据库
        logger.info("正在清空数据库...")
        importer.clean_database()
        
        # 导入所有节点
        logger.info("开始导入节点...")
        for file_name, label in node_files:
            file_path = os.path.join(file_dir, file_name)
            importer.import_nodes(file_path, label)
        
        # 导入所有关系
        logger.info("开始导入关系...")
        for config in relation_files:
            file_path = os.path.join(file_dir, config['file'])
            importer.import_relations(
                file_path,
                config['rel_type'],
                config['start_label'],
                config['end_label'],
                config['start_id'],
                config['end_id'],
                config['extra_props']
            )
        
        logger.info("数据导入完成！")
    
    except Exception as e:
        logger.error(f"导入过程中出错: {str(e)}")
    finally:
        importer.close()

if __name__ == "__main__":
    main()

2025-08-05 18:53:46,637 - INFO - 正在清空数据库...
2025-08-05 18:53:49,065 - INFO - 数据库已清空
2025-08-05 18:53:49,066 - INFO - 开始导入节点...
2025-08-05 18:53:49,709 - INFO - 导入节点: ./supplier_nodes.csv -> Supplier (16 行)
2025-08-05 18:53:50,359 - INFO - 导入节点: ./shipper_nodes.csv -> Shipper (11 行)
2025-08-05 18:53:51,006 - INFO - 导入节点: ./review_nodes.csv -> Review (31 行)
2025-08-05 18:53:51,646 - INFO - 导入节点: ./product_nodes.csv -> Product (11 行)
2025-08-05 18:53:52,542 - INFO - 导入节点: ./order_nodes.csv -> Order (101 行)
2025-08-05 18:53:53,184 - INFO - 导入节点: ./employee_nodes.csv -> Employee (4 行)
2025-08-05 18:53:53,828 - INFO - 导入节点: ./customer_nodes.csv -> Customer (21 行)
2025-08-05 18:53:54,887 - INFO - 导入节点: ./category_nodes.csv -> Category (21 行)
2025-08-05 18:53:54,888 - INFO - 开始导入关系...
2025-08-05 18:53:54,889 - ERROR - 导入过程中出错: ':START_ID'


In [22]:
from py2neo import Graph, Node, Relationship, Subgraph
import pandas as pd
import os
import re

# -------------------- 连接 --------------------
uri = "neo4j+s://981d8a45.databases.neo4j.io"
user = "neo4j"
password = "7bhGR2TUaVYg3N72GVGSf2g8DP16R-tmOXa4Vid0PFw"          # 改成你自己的
graph = Graph(uri, auth=(user, password))

# 清空数据库
graph.delete_all()

# -------------------- 节点 --------------------
node_files = {
    "supplier_nodes.csv":   "Supplier",
    "shipper_nodes.csv":    "Shipper",
    "review_nodes.csv":     "Review",
    "product_nodes.csv":    "Product",
    "employee_nodes.csv":   "Employee",
    "customer_nodes.csv":   "Customer",
    "category_nodes.csv":   "Category"
}

csv_dir = "/root/autodl-tmp/kefu_2/graphrag_2.1.0/graphrag/origin_data/data/ceshi/"

for file_name, label in node_files.items():
    path = os.path.join(csv_dir, file_name)
    if not os.path.exists(path):
        print(f"⚠️ 文件不存在: {file_name}")
        continue
    df = pd.read_csv(path).fillna("")
    nodes = []
    for _, row in df.iterrows():
        node = Node(label)
        for col in df.columns:
            if col.startswith("labels:") or ":LABEL" in col:
                continue
            key = col.split(":")[0]   # 取冒号前部分
            node[key] = str(row[col])
        nodes.append(node)
    graph.create(Subgraph(nodes))
    print(f"✅ 已导入 {len(nodes)} 个 {label} 节点")

# -------------------- 通用建索引（可选，提升关系创建速度） --------------------
graph.run("CREATE INDEX IF NOT EXISTS FOR (p:Product)   ON (p.productId)")
graph.run("CREATE INDEX IF NOT EXISTS FOR (s:Supplier)  ON (s.supplierId)")
graph.run("CREATE INDEX IF NOT EXISTS FOR (r:Review)    ON (r.reviewId)")
graph.run("CREATE INDEX IF NOT EXISTS FOR (o:Order)     ON (o.orderId)")
graph.run("CREATE INDEX IF NOT EXISTS FOR (c:Customer)  ON (c.customerId)")
graph.run("CREATE INDEX IF NOT EXISTS FOR (e:Employee)  ON (e.employeeId)")
graph.run("CREATE INDEX IF NOT EXISTS FOR (sh:Shipper)  ON (sh.shipperId)")
graph.run("CREATE INDEX IF NOT EXISTS FOR (ca:Category) ON (ca.categoryId)")

# -------------------- 关系 --------------------
edge_files = [
    "review_product_edges.csv",
    "product_supplier_edges.csv",
    "customer_order_edges.csv",
    "product_category_edges.csv",
    "order_shipper_edges.csv",
    "order_product_edges.csv",
    "employee_reports_to_edges.csv",
    "employee_order_edges.csv",
    "customer_review_edges.csv"
]

# 根据文件中的列名提取起始/结束 ID
id_col_pattern = re.compile(r":START_ID\(([^)]+)\)|:START_ID")
id_col_pattern_end = re.compile(r":END_ID\(([^)]+)\)|:END_ID")

for file_name in edge_files:
    path = os.path.join(csv_dir, file_name)
    if not os.path.exists(path):
        print(f"⚠️ 文件不存在: {file_name}")
        continue

    df = pd.read_csv(path).fillna("")
    # 找到起始/结束列名
    start_col = next(c for c in df.columns if id_col_pattern.match(c))
    end_col   = next(c for c in df.columns if id_col_pattern_end.match(c))
    rel_type_col = ":TYPE"

    # 根据列名推断节点标签
    m_start = id_col_pattern.match(start_col)
    m_end   = id_col_pattern_end.match(end_col)
    start_label = m_start.group(1) if m_start.group(1) else None
    end_label   = m_end.group(1)   if m_end.group(1)   else None

    # 如果列名未带括号，则通过文件内容猜（兜底）
    if not start_label:
        start_label = {
            "review_product": "Review",
            "product_supplier": "Product",
            "customer_order": "Customer",
            "product_category": "Product",
            "order_shipper": "Order",
            "order_product": "Order",
            "employee_reports_to": "Employee",
            "employee_order": "Employee",
            "customer_review": "Customer"
        }[file_name.replace(".csv", "").replace("_edges", "")]

    if not end_label:
        end_label = {
            "review_product": "Product",
            "product_supplier": "Supplier",
            "customer_order": "Order",
            "product_category": "Category",
            "order_shipper": "Shipper",
            "order_product": "Product",
            "employee_reports_to": "Employee",
            "employee_order": "Order",
            "customer_review": "Review"
        }[file_name.replace(".csv", "").replace("_edges", "")]

    start_label = start_label[0].upper() + start_label[1:]
    end_label   = end_label[0].upper()   + end_label[1:]

    # 小写 ID 字段名
    start_id_key = start_label.lower() + "Id"
    end_id_key   = end_label.lower() + "Id"

    tx = graph.begin()
    for _, row in df.iterrows():
        start_id = str(row[start_col])
        end_id   = str(row[end_col])
        rel_type = str(row[rel_type_col])

        start_node = graph.nodes.match(start_label).where(f"_.{start_id_key} = '{start_id}'").first()
        end_node   = graph.nodes.match(end_label).where(f"_.{end_id_key} = '{end_id}'").first()

        if start_node and end_node:
            tx.create(Relationship(start_node, rel_type, end_node))
        else:
            print(f"⚠️ 无法创建关系: {start_id} -[{rel_type}]-> {end_id}")
    tx.commit()
    print(f"✅ 已处理关系文件: {file_name}")

print("🎉 全部数据导入完成！")

✅ 已导入 15 个 Supplier 节点
✅ 已导入 10 个 Shipper 节点
✅ 已导入 30 个 Review 节点
✅ 已导入 10 个 Product 节点
✅ 已导入 3 个 Employee 节点
✅ 已导入 20 个 Customer 节点
✅ 已导入 20 个 Category 节点


/tmp/ipykernel_12950/3502768129.py:138: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


✅ 已处理关系文件: review_product_edges.csv


/tmp/ipykernel_12950/3502768129.py:138: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


✅ 已处理关系文件: product_supplier_edges.csv
⚠️ 无法创建关系: TB272 -[PLACED]-> 3
⚠️ 无法创建关系: XT154 -[PLACED]-> 5
⚠️ 无法创建关系: NU707 -[PLACED]-> 7
⚠️ 无法创建关系: QF269 -[PLACED]-> 14
⚠️ 无法创建关系: HR018 -[PLACED]-> 15
⚠️ 无法创建关系: GB963 -[PLACED]-> 20
⚠️ 无法创建关系: PK763 -[PLACED]-> 23
⚠️ 无法创建关系: AD754 -[PLACED]-> 25
⚠️ 无法创建关系: PK763 -[PLACED]-> 28
⚠️ 无法创建关系: PK763 -[PLACED]-> 29
⚠️ 无法创建关系: NU707 -[PLACED]-> 31
⚠️ 无法创建关系: AD754 -[PLACED]-> 35
⚠️ 无法创建关系: IS953 -[PLACED]-> 41
⚠️ 无法创建关系: AQ661 -[PLACED]-> 52
⚠️ 无法创建关系: PK763 -[PLACED]-> 61
⚠️ 无法创建关系: AQ661 -[PLACED]-> 64
⚠️ 无法创建关系: TB272 -[PLACED]-> 68
⚠️ 无法创建关系: PC981 -[PLACED]-> 69
⚠️ 无法创建关系: TB272 -[PLACED]-> 70
⚠️ 无法创建关系: YX989 -[PLACED]-> 74
⚠️ 无法创建关系: TB272 -[PLACED]-> 77
⚠️ 无法创建关系: PK763 -[PLACED]-> 78
⚠️ 无法创建关系: HR018 -[PLACED]-> 80
⚠️ 无法创建关系: QF269 -[PLACED]-> 81
⚠️ 无法创建关系: ON539 -[PLACED]-> 92
⚠️ 无法创建关系: AQ661 -[PLACED]-> 93
⚠️ 无法创建关系: GB963 -[PLACED]-> 94
⚠️ 无法创建关系: TT217 -[PLACED]-> 96
⚠️ 无法创建关系: OF968 -[PLACED]-> 2
⚠️ 无法创建关系: OZ939 -[PLACED]-> 4
⚠️ 无法创建

...

/tmp/ipykernel_12950/3502768129.py:138: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


✅ 已处理关系文件: order_product_edges.csv
⚠️ 无法创建关系: 2 -[REPORTS_TO]-> 1.0


/tmp/ipykernel_12950/3502768129.py:138: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  tx.commit()


✅ 已处理关系文件: employee_reports_to_edges.csv
⚠️ 无法创建关系: 1 -[PROCESSED]-> 3
⚠️ 无法创建关系: 1 -[PROCESSED]-> 5
⚠️ 无法创建关系: 1 -[PROCESSED]-> 7
⚠️ 无法创建关系: 1 -[PROCESSED]-> 14
⚠️ 无法创建关系: 1 -[PROCESSED]-> 15
⚠️ 无法创建关系: 1 -[PROCESSED]-> 20
⚠️ 无法创建关系: 1 -[PROCESSED]-> 23
⚠️ 无法创建关系: 1 -[PROCESSED]-> 25
⚠️ 无法创建关系: 1 -[PROCESSED]-> 28
⚠️ 无法创建关系: 1 -[PROCESSED]-> 29
⚠️ 无法创建关系: 1 -[PROCESSED]-> 31
⚠️ 无法创建关系: 1 -[PROCESSED]-> 35


KeyboardInterrupt: 

In [25]:
from neo4j import GraphDatabase
import csv
import os
from datetime import datetime
import logging

# 配置日志
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class Neo4jImporter:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def close(self):
        self.driver.close()
    
    def clean_database(self):
        """删除数据库中的所有节点和关系"""
        with self.driver.session() as session:
            query = "MATCH (n) DETACH DELETE n"
            session.run(query)
            logger.info("数据库已清空")
    
    def import_nodes(self, file_path, label):
        """导入节点数据"""
        with self.driver.session() as session:
            with open(file_path, 'r', encoding='utf-8') as file:
                reader = csv.DictReader(file)
                batch = []
                batch_size = 100
                
                for row in reader:
                    # 准备节点属性
                    properties = {}
                    for key, value in row.items():
                        # 处理特殊字段
                        if key == "BirthDate" or key == "HireDate":
                            properties[key] = datetime.strptime(value, '%Y-%m-%d').date()
                        elif key in ["OrderDate", "RequiredDate", "ShippedDate"]:
                            if value:  # 处理空日期
                                properties[key] = datetime.strptime(value, '%Y-%m-%d %H:%M:%S')
                        else:
                            properties[key] = value
                    
                    batch.append(properties)
                    
                    if len(batch) >= batch_size:
                        session.execute_write(self._create_nodes, batch, label)
                        batch = []
                
                if batch:
                    session.execute_write(self._create_nodes, batch, label)
                logger.info(f"导入节点: {file_path} -> {label} ({reader.line_num} 行)")
    
    @staticmethod
    def _create_nodes(tx, batch, label):
        query = f"""
        UNWIND $batch AS row
        CREATE (n:{label})
        SET n = row
        """
        tx.run(query, batch=batch)
    
    def import_relations(self, file_path, rel_type, start_label, end_label, 
                         start_id_prop, end_id_prop, extra_props=None):
        """导入关系数据"""
        with self.driver.session() as session:
            with open(file_path, 'r', encoding='utf-8') as file:
                # 读取第一行获取实际列名
                header_line = file.readline().strip()
                file.seek(0)  # 重置文件指针
                
                # 创建reader对象
                reader = csv.DictReader(file)
                
                # 确定实际列名
                start_id_col = None
                end_id_col = None
                type_col = None
                
                for col in reader.fieldnames:
                    if col.startswith(':START_ID('):
                        start_id_col = col
                    elif col.startswith(':END_ID('):
                        end_id_col = col
                    elif col == ':TYPE':
                        type_col = col
                
                if not start_id_col or not end_id_col:
                    logger.error(f"在文件 {file_path} 中找不到START_ID或END_ID列")
                    return
                
                batch = []
                batch_size = 100
                
                for row in reader:
                    # 准备关系属性
                    rel_properties = {}
                    if extra_props:
                        for prop in extra_props:
                            if prop in row:
                                # 转换数值类型
                                if prop in ["UnitPrice", "Discount", "Freight", "Rating"]:
                                    rel_properties[prop] = float(row[prop])
                                elif prop in ["Quantity"]:
                                    rel_properties[prop] = int(row[prop])
                                else:
                                    rel_properties[prop] = row[prop]
                    
                    # 创建关系对象
                    relation = {
                        'start_id': row[start_id_col],
                        'end_id': row[end_id_col],
                        'properties': rel_properties
                    }
                    batch.append(relation)
                    
                    if len(batch) >= batch_size:
                        session.execute_write(
                            self._create_relations, 
                            batch, 
                            rel_type, 
                            start_label, 
                            end_label,
                            start_id_prop,
                            end_id_prop
                        )
                        batch = []
                
                if batch:
                    session.execute_write(
                        self._create_relations, 
                        batch, 
                        rel_type, 
                        start_label, 
                        end_label,
                        start_id_prop,
                        end_id_prop
                    )
                logger.info(f"导入关系: {file_path} -> {rel_type} ({reader.line_num} 行)")
    
    @staticmethod
    def _create_relations(tx, batch, rel_type, start_label, end_label, start_id_prop, end_id_prop):
        query = f"""
        UNWIND $batch AS rel
        MATCH (start:{start_label} {{{start_id_prop}: rel.start_id}})
        MATCH (end:{end_label} {{{end_id_prop}: rel.end_id}})
        CREATE (start)-[r:{rel_type}]->(end)
        SET r = rel.properties
        """
        tx.run(query, batch=batch)

def main():
    # Neo4j数据库连接配置
    uri = "neo4j+s://981d8a45.databases.neo4j.io"  # 修改为你的Neo4j地址
    user = "neo4j"                # 修改为你的用户名
    password = "7bhGR2TUaVYg3N72GVGSf2g8DP16R-tmOXa4Vid0PFw"         # 修改为你的密码
    
    # 文件路径配置（假设所有文件在当前目录）
    file_dir = './'
    
    # 节点文件配置
    node_files = [
        ('supplier_nodes.csv', 'Supplier'),
        ('shipper_nodes.csv', 'Shipper'),
        ('review_nodes.csv', 'Review'),
        ('product_nodes.csv', 'Product'),
        ('order_nodes.csv', 'Order'),
        ('employee_nodes.csv', 'Employee'),
        ('customer_nodes.csv', 'Customer'),
        ('category_nodes.csv', 'Category'),
    ]
    
    # 关系文件配置
    relation_files = [
        {
            'file': 'review_product_edges.csv',
            'rel_type': 'ABOUT',
            'start_label': 'Review',
            'end_label': 'Product',
            'start_id': 'reviewId',
            'end_id': 'productId',
            'extra_props': None
        },
        {
            'file': 'product_supplier_edges.csv',
            'rel_type': 'SUPPLIED_BY',
            'start_label': 'Product',
            'end_label': 'Supplier',
            'start_id': 'productId',
            'end_id': 'supplierId',
            'extra_props': None
        },
        {
            'file': 'customer_order_edges.csv',
            'rel_type': 'PLACED',
            'start_label': 'Customer',
            'end_label': 'Order',
            'start_id': 'customerId',
            'end_id': 'orderId',
            'extra_props': None
        },
        {
            'file': 'product_category_edges.csv',
            'rel_type': 'BELONGS_TO',
            'start_label': 'Product',
            'end_label': 'Category',
            'start_id': 'productId',
            'end_id': 'categoryId',
            'extra_props': None
        },
        {
            'file': 'order_shipper_edges.csv',
            'rel_type': 'SHIPPED_VIA',
            'start_label': 'Order',
            'end_label': 'Shipper',
            'start_id': 'orderId',
            'end_id': 'shipperId',
            'extra_props': None
        },
        {
            'file': 'order_product_edges.csv',
            'rel_type': 'CONTAINS',
            'start_label': 'Order',
            'end_label': 'Product',
            'start_id': 'orderId',
            'end_id': 'productId',
            'extra_props': ['UnitPrice', 'Quantity', 'Discount']
        },
        {
            'file': 'employee_reports_to_edges.csv',
            'rel_type': 'REPORTS_TO',
            'start_label': 'Employee',
            'end_label': 'Employee',
            'start_id': 'employeeId',
            'end_id': 'employeeId',
            'extra_props': None
        },
        {
            'file': 'employee_order_edges.csv',
            'rel_type': 'PROCESSED',
            'start_label': 'Employee',
            'end_label': 'Order',
            'start_id': 'employeeId',
            'end_id': 'orderId',
            'extra_props': None
        },
        {
            'file': 'customer_review_edges.csv',
            'rel_type': 'WROTE',
            'start_label': 'Customer',
            'end_label': 'Review',
            'start_id': 'customerId',
            'end_id': 'reviewId',
            'extra_props': None
        }
    ]

    importer = Neo4jImporter(uri, user, password)
    
    try:
        # 清空数据库
        logger.info("正在清空数据库...")
        importer.clean_database()
        
        # 导入所有节点
        logger.info("开始导入节点...")
        for file_name, label in node_files:
            file_path = os.path.join(file_dir, file_name)
            if os.path.exists(file_path):
                importer.import_nodes(file_path, label)
            else:
                logger.error(f"文件不存在: {file_path}")
        
        # 导入所有关系
        logger.info("开始导入关系...")
        for config in relation_files:
            file_path = os.path.join(file_dir, config['file'])
            if os.path.exists(file_path):
                print("进来了吗？")
                importer.import_relations(
                    file_path,
                    config['rel_type'],
                    config['start_label'],
                    config['end_label'],
                    config['start_id'],
                    config['end_id'],
                    config['extra_props']
                )
            else:
                logger.error(f"文件不存在: {file_path}")
        
        logger.info("数据导入完成！")
    
    except Exception as e:
        logger.error(f"导入过程中出错: {str(e)}", exc_info=True)
    finally:
        importer.close()

if __name__ == "__main__":
    main()

2025-08-05 19:08:49,564 - INFO - 正在清空数据库...
2025-08-05 19:08:51,803 - INFO - 数据库已清空
2025-08-05 19:08:51,835 - INFO - 开始导入节点...
2025-08-05 19:08:52,460 - INFO - 导入节点: ./supplier_nodes.csv -> Supplier (16 行)
2025-08-05 19:08:53,080 - INFO - 导入节点: ./shipper_nodes.csv -> Shipper (11 行)
2025-08-05 19:08:53,706 - INFO - 导入节点: ./review_nodes.csv -> Review (31 行)
2025-08-05 19:08:54,332 - INFO - 导入节点: ./product_nodes.csv -> Product (11 行)
2025-08-05 19:08:55,217 - INFO - 导入节点: ./order_nodes.csv -> Order (101 行)
2025-08-05 19:08:55,838 - INFO - 导入节点: ./employee_nodes.csv -> Employee (4 行)
2025-08-05 19:08:56,465 - INFO - 导入节点: ./customer_nodes.csv -> Customer (21 行)
2025-08-05 19:08:57,745 - INFO - 导入节点: ./category_nodes.csv -> Category (21 行)
2025-08-05 19:08:57,746 - INFO - 开始导入关系...


进来了吗？


2025-08-05 19:08:58,386 - INFO - 导入关系: ./review_product_edges.csv -> ABOUT (31 行)


进来了吗？


2025-08-05 19:08:59,016 - INFO - 导入关系: ./product_supplier_edges.csv -> SUPPLIED_BY (11 行)


...

2025-08-05 19:09:04,286 - INFO - 导入关系: ./employee_reports_to_edges.csv -> REPORTS_TO (2 行)


进来了吗？


2025-08-05 19:09:04,917 - INFO - 导入关系: ./employee_order_edges.csv -> PROCESSED (101 行)


进来了吗？


2025-08-05 19:09:05,551 - INFO - 导入关系: ./customer_review_edges.csv -> WROTE (31 行)
2025-08-05 19:09:05,552 - INFO - 数据导入完成！


In [27]:
from py2neo import Graph, Node, Subgraph
import pandas as pd
import os

URI  = "neo4j+s://981d8a45.databases.neo4j.io"
USER = "neo4j"
PWD  = "7bhGR2TUaVYg3N72GVGSf2g8DP16R-tmOXa4Vid0PFw"      # ← 改成你自己的
graph = Graph(URI, auth=(USER, PWD))

# 1️⃣ 清空数据库
graph.run("MATCH (n) DETACH DELETE n")

# 2️⃣ 读取全部节点，统一主键字段为 id
node_files = {
    "supplier_nodes.csv": "Supplier",
    "shipper_nodes.csv":  "Shipper",
    "review_nodes.csv":   "Review",
    "product_nodes.csv":  "Product",
    "employee_nodes.csv": "Employee",
    "customer_nodes.csv": "Customer",
    "category_nodes.csv": "Category"
}

csv_dir = "/root/autodl-tmp/kefu_2/graphrag_2.1.0/graphrag/origin_data/data/ceshi/"

# 缓存：{label: {id: node}}
node_cache = {}

for file_name, label in node_files.items():
    path = os.path.join(csv_dir, file_name)
    if not os.path.exists(path):
        continue
    df = pd.read_csv(path).fillna("")
    nodes = []
    for _, row in df.iterrows():
        # 找出主键列
        id_col = next(c for c in df.columns if ":ID" in c)
        nid = str(row[id_col])
        props = {"id": nid}
        for col in df.columns:
            if ":ID" in col or ":LABEL" in col or col == "labels:LABEL":
                continue
            key = col.split(":")[0]
            props[key] = str(row[col])
        node = Node(label, **props)
        nodes.append(node)
    subgraph = Subgraph(nodes)
    graph.create(subgraph)
    # 填充缓存
    node_cache[label] = {n["id"]: n for n in nodes}
    print(f"✅ {label}: {len(nodes)} 节点已导入")

# 3️⃣ 读取全部 edges，批量创建关系
edge_files = [
    ("review_product_edges.csv",      "Review",   "Product",  "ABOUT"),
    ("product_supplier_edges.csv",    "Product",  "Supplier", "SUPPLIED_BY"),
    ("customer_order_edges.csv",      "Customer", "Order",    "PLACED"),
    ("product_category_edges.csv",    "Product",  "Category", "BELONGS_TO"),
    ("order_shipper_edges.csv",       "Order",    "Shipper",  "SHIPPED_VIA"),
    ("order_product_edges.csv",       "Order",    "Product",  "CONTAINS"),
    ("employee_reports_to_edges.csv", "Employee", "Employee", "REPORTS_TO"),
    ("employee_order_edges.csv",      "Employee", "Order",    "PROCESSED"),
    ("customer_review_edges.csv",     "Customer", "Review",   "WROTE")
]

for file_name, start_label, end_label, default_type in edge_files:
    path = os.path.join(csv_dir, file_name)
    if not os.path.exists(path):
        continue
    df = pd.read_csv(path).fillna("")

    # 列名不一定是 :START_ID / :END_ID
    start_col = next(c for c in df.columns if ":START" in c)
    end_col   = next(c for c in df.columns if ":END" in c)
    type_col  = ":TYPE" if ":TYPE" in df.columns else None

    rels = []
    for _, row in df.iterrows():
        sid  = str(row[start_col])
        eid  = str(row[end_col])
        rtyp = str(row[type_col]) if type_col else default_type
        s_node = node_cache[start_label].get(sid)
        e_node = node_cache[end_label].get(eid)
        if s_node and e_node:
            rels.append({"sid": sid, "eid": eid, "type": rtyp})

    if rels:
        graph.run(
            """
            UNWIND $rels AS rel
            MATCH (a {id: rel.sid}), (b {id: rel.eid})
            CALL apoc.merge.relationship(a, rel.type, {}, {}, b) YIELD rel
            RETURN count(rel)
            """,
            rels=rels
        )
    print(f"✅ {file_name}: {len(rels)} 条关系已创建")

print("🎉 全部节点和关系导入完成！")

✅ Supplier: 15 节点已导入
✅ Shipper: 10 节点已导入
✅ Review: 30 节点已导入
✅ Product: 10 节点已导入
✅ Employee: 3 节点已导入
✅ Customer: 20 节点已导入
✅ Category: 20 节点已导入


ClientError: [Statement.SyntaxError] Variable `rel` already declared (line 4, column 72 (offset: 159))
"            CALL apoc.merge.relationship(a, rel.type, {}, {}, b) YIELD rel"
                                                                        ^

In [28]:
from py2neo import Graph, Node, Subgraph
import pandas as pd
import os

# ---------------- 配置 ----------------
URI  = "neo4j+s://981d8a45.databases.neo4j.io"
USER = "neo4j"
PWD  = "7bhGR2TUaVYg3N72GVGSf2g8DP16R-tmOXa4Vid0PFw"          # ← 改成你自己的
graph = Graph(URI, auth=(USER, PWD))

# 1️⃣ 清空数据库
graph.run("MATCH (n) DETACH DELETE n")

# 2️⃣ 节点文件映射
node_files = {
    "supplier_nodes.csv": "Supplier",
    "shipper_nodes.csv":  "Shipper",
    "review_nodes.csv":   "Review",
    "product_nodes.csv":  "Product",
    "employee_nodes.csv": "Employee",
    "customer_nodes.csv": "Customer",
    "category_nodes.csv": "Category"
}

csv_dir = "/root/autodl-tmp/kefu_2/graphrag_2.1.0/graphrag/origin_data/data/ceshi/"

# 缓存：label -> {id -> node}
node_cache = {}

for file_name, label in node_files.items():
    path = os.path.join(csv_dir, file_name)
    if not os.path.exists(path):
        continue
    df = pd.read_csv(path).fillna("")
    nodes = []
    for _, row in df.iterrows():
        id_col = next(c for c in df.columns if ":ID" in c)
        nid = str(row[id_col])
        props = {"id": nid}
        for col in df.columns:
            if ":ID" in col or ":LABEL" in col or col == "labels:LABEL":
                continue
            key = col.split(":")[0]
            props[key] = str(row[col])
        node = Node(label, **props)
        nodes.append(node)
    graph.create(Subgraph(nodes))
    node_cache[label] = {n["id"]: n for n in nodes}
    print(f"✅ {label}: {len(nodes)} 节点已导入")

# 3️⃣ 边文件配置：文件名、(起始标签, 结束标签, 默认关系类型)
edge_files = [
    ("review_product_edges.csv",      "Review",   "Product",  "ABOUT"),
    ("product_supplier_edges.csv",    "Product",  "Supplier", "SUPPLIED_BY"),
    ("customer_order_edges.csv",      "Customer", "Order",    "PLACED"),
    ("product_category_edges.csv",    "Product",  "Category", "BELONGS_TO"),
    ("order_shipper_edges.csv",       "Order",    "Shipper",  "SHIPPED_VIA"),
    ("order_product_edges.csv",       "Order",    "Product",  "CONTAINS"),
    ("employee_reports_to_edges.csv", "Employee", "Employee", "REPORTS_TO"),
    ("employee_order_edges.csv",      "Employee", "Order",    "PROCESSED"),
    ("customer_review_edges.csv",     "Customer", "Review",   "WROTE")
]

for file_name, start_label, end_label, default_type in edge_files:
    path = os.path.join(csv_dir, file_name)
    if not os.path.exists(path):
        continue
    df = pd.read_csv(path).fillna("")
    start_col = next(c for c in df.columns if ":START" in c)
    end_col   = next(c for c in df.columns if ":END" in c)
    type_col  = ":TYPE" if ":TYPE" in df.columns else None

    rels = []
    for _, row in df.iterrows():
        sid  = str(row[start_col])
        eid  = str(row[end_col])
        rtyp = str(row[type_col]) if type_col else default_type
        s_node = node_cache[start_label].get(sid)
        e_node = node_cache[end_label].get(eid)
        if s_node and e_node:
            rels.append({"sid": sid, "eid": eid, "type": rtyp})

    if rels:
        graph.run(
            """
            UNWIND $rels AS r
            MATCH (a {id: r.sid}), (b {id: r.eid})
            CALL apoc.merge.relationship(a, r.type, {}, {}, b) YIELD rel as ignored
            RETURN count(ignored)
            """,
            rels=rels
        )
    print(f"✅ {file_name}: {len(rels)} 条关系已创建")

print("🎉 全部节点与关系导入完成！")

✅ Supplier: 15 节点已导入
✅ Shipper: 10 节点已导入
✅ Review: 30 节点已导入
✅ Product: 10 节点已导入
✅ Employee: 3 节点已导入
✅ Customer: 20 节点已导入
✅ Category: 20 节点已导入
✅ review_product_edges.csv: 30 条关系已创建
✅ product_supplier_edges.csv: 10 条关系已创建


KeyError: 'Order'

In [29]:
from py2neo import Graph, Node, Subgraph
import pandas as pd
import os

# ---------------- 配置 ----------------
URI  = "neo4j+s://981d8a45.databases.neo4j.io"
USER = "neo4j"
PWD  = "7bhGR2TUaVYg3N72GVGSf2g8DP16R-tmOXa4Vid0PFw"   
graph = Graph(URI, auth=(USER, PWD))

# 1️⃣ 清空数据库
graph.run("MATCH (n) DETACH DELETE n")

# 2️⃣ 节点文件映射（**已补充 order_nodes.csv**）
node_files = {
    "supplier_nodes.csv": "Supplier",
    "shipper_nodes.csv":  "Shipper",
    "review_nodes.csv":   "Review",
    "product_nodes.csv":  "Product",
    "employee_nodes.csv": "Employee",
    "customer_nodes.csv": "Customer",
    "category_nodes.csv": "Category",
    "order_nodes.csv":    "Order"        # ← 新增
}

csv_dir = "/root/autodl-tmp/kefu_2/graphrag_2.1.0/graphrag/origin_data/data/ceshi/"
node_cache = {}     # label -> {id -> node}

# ---------- 通用读节点 ----------
for file_name, label in node_files.items():
    path = os.path.join(csv_dir, file_name)
    if not os.path.exists(path):
        continue
    df = pd.read_csv(path).fillna("")
    nodes = []
    for _, row in df.iterrows():
        id_col = next(c for c in df.columns if ":ID" in c)
        nid = str(row[id_col])
        props = {"id": nid}
        for col in df.columns:
            if ":ID" in col or ":LABEL" in col or col == "labels:LABEL":
                continue
            key = col.split(":")[0]
            props[key] = str(row[col])
        node = Node(label, **props)
        nodes.append(node)
    graph.create(Subgraph(nodes))
    node_cache[label] = {n["id"]: n for n in nodes}
    print(f"✅ {label}: {len(nodes)} 节点已导入")

# 3️⃣ 边文件配置
edge_files = [
    ("review_product_edges.csv",      "Review",   "Product",  "ABOUT"),
    ("product_supplier_edges.csv",    "Product",  "Supplier", "SUPPLIED_BY"),
    ("customer_order_edges.csv",      "Customer", "Order",    "PLACED"),
    ("product_category_edges.csv",    "Product",  "Category", "BELONGS_TO"),
    ("order_shipper_edges.csv",       "Order",    "Shipper",  "SHIPPED_VIA"),
    ("order_product_edges.csv",       "Order",    "Product",  "CONTAINS"),
    ("employee_reports_to_edges.csv", "Employee", "Employee", "REPORTS_TO"),
    ("employee_order_edges.csv",      "Employee", "Order",    "PROCESSED"),
    ("customer_review_edges.csv",     "Customer", "Review",   "WROTE")
]

# ---------- 通用读边 ----------
for file_name, start_lbl, end_lbl, default_rel in edge_files:
    path = os.path.join(csv_dir, file_name)
    if not os.path.exists(path):
        continue
    df = pd.read_csv(path).fillna("")
    start_col = next(c for c in df.columns if ":START" in c)
    end_col   = next(c for c in df.columns if ":END" in c)
    type_col  = ":TYPE" if ":TYPE" in df.columns else None

    rels = []
    for _, row in df.iterrows():
        sid  = str(row[start_col])
        eid  = str(row[end_col])
        rtyp = str(row[type_col]) if type_col else default_rel
        # 用缓存字典直接取节点
        s_node = node_cache[start_lbl].get(sid)
        e_node = node_cache[end_lbl].get(eid)
        if s_node and e_node:
            rels.append({"sid": sid, "eid": eid, "type": rtyp})

    if rels:
        graph.run(
            """
            UNWIND $rels AS r
            MATCH (a {id: r.sid}), (b {id: r.eid})
            CALL apoc.merge.relationship(a, r.type, {}, {}, b) YIELD rel as _
            RETURN count(_)
            """,
            rels=rels
        )
    print(f"✅ {file_name}: {len(rels)} 条关系已创建")

print("🎉 全部节点与关系导入完成！")

✅ Supplier: 15 节点已导入
✅ Shipper: 10 节点已导入
✅ Review: 30 节点已导入
✅ Product: 10 节点已导入
✅ Employee: 3 节点已导入
✅ Customer: 20 节点已导入
✅ Category: 20 节点已导入
✅ Order: 100 节点已导入
✅ review_product_edges.csv: 30 条关系已创建
✅ product_supplier_edges.csv: 10 条关系已创建
✅ customer_order_edges.csv: 100 条关系已创建
✅ product_category_edges.csv: 10 条关系已创建
✅ order_shipper_edges.csv: 100 条关系已创建
✅ order_product_edges.csv: 303 条关系已创建
✅ employee_reports_to_edges.csv: 0 条关系已创建
✅ employee_order_edges.csv: 100 条关系已创建
✅ customer_review_edges.csv: 30 条关系已创建
🎉 全部节点与关系导入完成！
